In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt

data = str(datetime.now().year)+'-'+str(datetime.now().month)+'-'+str(datetime.now().day)
data

'2020-4-9'

In [7]:
corona_municipio =  pd.read_csv('data\\covid19_municipios.csv')
corona_municipio.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-03,AC,Acrelândia,city,9,0.0,True,15256.0,1200013.0,58.99318,NaN
1,2020-04-03,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-04-03,AC,Rio Branco,city,36,0.0,True,407319.0,1200401.0,8.83828,NaN
3,2020-04-03,AC,NaN,state,46,0.0,True,881935.0,12.0,5.21580,NaN
4,2020-04-03,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN


In [8]:
corona_municipio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5155 entries, 0 to 5154
Data columns (total 11 columns):
date                              5155 non-null object
state                             5155 non-null object
city                              4607 non-null object
place_type                        5155 non-null object
confirmed                         5155 non-null int64
deaths                            4825 non-null float64
is_last                           5155 non-null bool
estimated_population_2019         5009 non-null float64
city_ibge_code                    5009 non-null float64
confirmed_per_100k_inhabitants    4962 non-null float64
death_rate                        549 non-null float64
dtypes: bool(1), float64(5), int64(1), object(4)
memory usage: 327.3+ KB


In [9]:
corona_municipio['date'] = pd.to_datetime(corona_municipio['date'])
corona_municipio.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-03,AC,Acrelândia,city,9,0.0,True,15256.0,1200013.0,58.99318,NaN
1,2020-04-03,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-04-03,AC,Rio Branco,city,36,0.0,True,407319.0,1200401.0,8.83828,NaN
3,2020-04-03,AC,NaN,state,46,0.0,True,881935.0,12.0,5.21580,NaN
4,2020-04-03,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN


In [14]:
df = pd.DataFrame()
group = corona_municipio.groupby('date')
df['confirmed'] = group.sum()['confirmed']
df.tail()

,confirmed
date,
2020-03-30,9320
2020-03-31,11648
2020-04-01,13862
2020-04-02,16196
2020-04-03,17108
